In [34]:
import math
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
%matplotlib inline

In [35]:
def load_data(file, num):
    #导入数据
    #file表示文件,num表示需要取的列数
    
    f = open(file,mode = "r")  # 打开文件
    data = []
    for line in f.readlines():
        line_temp = line.split()
        data_temp = []
        for i in range(num):
            data_temp.append(line_temp[i])
        data.append(data_temp)
    f.close()  
    return data
        

In [36]:
data_2D = load_data("point.txt",2)
data_3D = load_data("point.txt",3)
data = load_data("point1.txt",3)

x=np.empty((len(data),1))
y=np.empty((len(data),1)) 
z=np.empty((len(data),1))
#point1.txt里只有一千多个点，取少量点画图看起来清晰一些

for i in range(len(data)):  #小部分数据
    x[i][0] = data[i][0]
    y[i][0] = data[i][1]
    z[i][0] = data[i][2]

In [37]:
x_data=np.empty((len(data_3D),1))  #全部三列数据
y_data=np.empty((len(data_3D),1))
z_data=np.empty((len(data_3D),1))
for i in range(len(data_3D)):
    x_data[i][0] = data_3D[i][0]
    y_data[i][0] = data_3D[i][1]
    z_data[i][0] = data_3D[i][2]


In [38]:
def draw_point2D(x,y): #x,y为numpy数组
    #画二维点云图
    plt.figure(figsize=(150,150))
    plt.plot(x, y, 'o', color='k')


In [39]:
def draw_point3D(x,y,z):
    #画三维点云图
    ax = plt.axes(projection='3d')
    ax.scatter3D(x,y,z,s=0.002,c="gray")
    ax.set_xlim(-9,9)
    ax.set_ylim(-10,10)
    ax.set_zlim(-10,15)

In [40]:
def cal_distance(a, b):  #求两点之间的距离
    sum = 0
    for i in range(len(a)):
        sum += np.sqrt((float(a[i])-float(b[i]))**2)
    return sum


In [41]:
def near_points(data, p_centroid, distance): #求给定半径内的点
    eligible_point = []
    for each in data:
        bet_distance = cal_distance(each, p_centroid)
        if bet_distance <= distance:
            eligible_point.append(each)
    return eligible_point


In [42]:
def gaussian_kernel(distance, sigma): #计算高斯核函数的值
    w = (1/(sigma*np.sqrt(2*np.pi))) * np.exp(-0.5*((distance / sigma))**2)
    return w


In [43]:
def plot_circle(center,r): #画出每次迭代的圈

    x = np.linspace(float(center[0]) - r, float(center[0]) + r, 4000)

    y1 = np.sqrt(r**2 - (x-float(center[0]))**2) + float(center[1])

    y2 = -np.sqrt(r**2 - (x-float(center[0]))**2) + float(center[1])

    plt.plot(x, y1, c='k')
    plt.plot(x, y2, c='k')
    plt.axis('equal')

In [44]:
def mean_shift3D(data,mid_p):

    num_iterations = 100  #迭代次数
    
    x = np.empty((num_iterations,1))
    y = np.empty((num_iterations,1))
    z = np.empty((num_iterations,1))
    
    top_distance = 1 # 给定半径
    sigma = 250  # 设置高斯核函数的带宽
    
    ax = plt.axes(projection='3d')
    ax.scatter3D(x_data,y_data,z_data,s=0.0002)
    mid_p = np.array(mid_p,dtype = "float64")
    ax.scatter3D(mid_p[0], mid_p[1], mid_p[2],s=100,cmap='Greens') 
    #画出三维点云

    
    for i in range(num_iterations):
        
        near = near_points(data_3D, mid_p, top_distance)
        
        numerator = 0 #下一次迭代球心坐标的分子
        denominator = 0 #下一次迭代球心坐标的分母
        
        for each in near: #计算下一次迭代的圆心
            distance = cal_distance(each, mid_p)
            w = gaussian_kernel(distance, sigma) #权重
           
            each = np.array(each,dtype = "float64")#统一数据类型，不然与w相乘会报错
            
            numerator += w * each
            denominator += w
        
        last_p = mid_p
        new_p = numerator / denominator
        mid_p=new_p
        x[i][0]=mid_p[0]
        y[i][0]=mid_p[1]
        z[i][0]=mid_p[2]
        
        
    ax.scatter3D(x, y, z,s=100,cmap='Greens') #画出所有点和第一次迭代的圆
        

    return mid_p

In [13]:
def mean_shift2D(data,mid_p):
    top_distance = 0.5 # 给定半径
    sigma = 25  # 设置高斯核函数的带宽
    
    draw_point2D(x_data,y_data)
    plot_circle(mid_p,top_distance) #画出所有点和第一次迭代的圆
    
    num_iterations = 100  #迭代次数
    
    for i in range(num_iterations):
        
        near = near_points(data_2D, mid_p, top_distance)
        
        numerator = 0 #下一次迭代圆心坐标的分子
        denominator = 0 #下一次迭代圆心坐标的分母
        
        for each in near: #计算下一次迭代的圆心
            distance = cal_distance(each, mid_p)
            w = gaussian_kernel(distance, sigma) #权重
           
            each = np.array(each,dtype = "float64")#统一数据类型，不然与w相乘会报错
            
            numerator += w * each
            denominator += w
        
        last_p = mid_p
        new_p = numerator / denominator
        mid_p=new_p
        
        plot_circle(mid_p,top_distance) #画出下一次迭代的圆
        if i != 0:
            plt.plot([last_p[0],new_p[0]],[last_p[1],new_p[1]],c="k") #连接两次迭代的圆心,i=0时会多出一条线，因此i不取0

    return mid_p

    